# Google Colab Project: Music Composition with BERT

## Introduction

In this project, we explore the fascinating world of music composition using state-of-the-art natural language processing models, specifically BERT (Bidirectional Encoder Representations from Transformers). The goal is to train the model to generate music compositions in ABC notation.

## Project Workflow

### Data Loading and Preparation

- **Data Source**: The project begins by obtaining a dataset of music compositions in ABC notation. This dataset contains the music pieces that we'll use for training our models.

- **Data Preprocessing**: The dataset is preprocessed to clean and format the ABC notation for model training. This includes tokenization and encoding into a suitable format for the models.

### BERT Model Training


- **Model Selection**: We also train a BERT model, adapted for music generation, using PyTorch.

- **Hyperparameter Tuning**: The training process includes hyperparameter tuning to find the optimal combination of settings for BERT.


In [1]:
import torch
from tqdm import tqdm
from argparse import ArgumentParser

import glob
import os
import pandas as pd

import sys
!pip install wandb

import wandb
wandb.login(key='30b44f6f59b06faebb3d1f78df32c6fd9961f07d')
!{sys.executable} -m pip install youtokentome
!{sys.executable} -m pip install transformers
!pip install accelerate -U
from transformers import Trainer, TrainingArguments,default_data_collator
import youtokentome as yttm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=d506834138c71cd24ea30e460873ecc99a4e53f30c635de31d97b063917a0721
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for youtokentome: filename=youtokentome-1.0.6-cp310-cp310-linux_x86_64.whl size=1948600 sha256=814a8d6cbc3fc708c7399f01decf0c7e62c359bb09f6a5e8a12b7b327bff4d9e
  Stored in directory: /root/.cache/pip/wheels/df/85/f8/301d2ba45f43f30bed2fe413efa760bc726b8b660ed9c2900c
Successfully built youtokentome
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.6 MB/s eta 0:00:00


In [2]:
ORIGIN = os.path.normpath(os.getcwd())
print(ORIGIN)
TRAIN_DIR ="/content/drive/MyDrive/test2/"
VALID_DIR = "/content/drive/MyDrive/Music_project/valid_path/"
TEST_DIR = "/content/drive/MyDrive/Music_project/test_path/"
TOKENIZER_DIR = "/content/drive/MyDrive/Music_project/abc_run5.yttm"
DATASET_DIR ="/content/drive/MyDrive/Music_project/300,000_new_samples.csv"
# OUTPUT_DIR = "/content/drive/MyDrive/Music_project/output_GPT2_checkpoints6"
OUTPUT_DIR = "/content/drive/MyDrive/Music_project/"


/content


In [ ]:
print("Loading tokenizer...")
tokenizer = yttm.BPE(TOKENIZER_DIR) # import the trained tokenizer



Loading tokenizer...


In [ ]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

def get_model(vocab_size=30000):
    config_encoder = BertConfig()
    config_decoder = BertConfig()

    config_encoder.vocab_size = vocab_size
    config_decoder.vocab_size = vocab_size

    config_decoder.is_decoder = True
    config_decoder.add_cross_attention = True

    config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
    model = EncoderDecoderModel(config=config)

    return model

model = get_model(vocab_size=tokenizer.vocab_size()) # load the BERT model



In [5]:
USEABLE_PARAMS = [i+":" for i in "BCDFGHIKLMmNOPQRrSsTUVWwXZ"] # These are the parameters for key

def read_abc(path):
    keys = []
    notes = []
    with open(path) as rf:
        for line in rf:
            line = line.strip()
            if line.startswith("%"): # Skip any commments
                continue

            if any([line.startswith(key) for key in USEABLE_PARAMS]):
                if(line.startswith('T')):
                    continue # skipping the title for better tokenization
#                 if(line.startswith('L')):
#                     print(line) ## Checking all L in all files
                # After checking the all midi files, they all have the length (L) : 1/8
                keys.append(line)
            else:
                notes.append(line)

    keys = " ".join(keys)

    notes = "".join(notes).strip()
    notes = notes.replace(" ", "")

    if notes.endswith("|"):
        notes = notes[:-1]
    # Remove unneeded character.
    notes = notes.replace(" \ ", "")
    notes = notes.replace("\\", "")
    notes = notes.replace("\ ", "")
    notes = notes.replace("x8|", "") # 8 because all of the midi file has a L:1/8 that means one muted bar
    notes = notes.replace("z8|", "") # 8 because all of the midi file has a L:1/8 that means one muted bar

    notes = notes.strip()
    notes = " ".join(notes.split(" "))

    if not keys or not notes:
        return None, None

    return keys, notes



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

OUTPUT_DIR

'/content/drive/MyDrive/Music_project/output_BERT_checkpoints6'

In [ ]:
def load_dataset(path):
  data = []
  count = 0
  counter = 0
  directory_files = os.listdir(path)
  directory_path = path

  for file in directory_files:
      filename = os.path.join(directory_path, file)
      print(filename)
      keys, notes = read_abc(filename)
      print("======================")
      print(keys)
      print(notes)
      if keys is None:
          continue

      keys_tokens = tokenizer.encode(keys)


      bars = notes.split(",")
      input_bars = []
      target_bars = []
      count = 0
      notes_tokens = [tokenizer.encode(i + " | ") for i in bars]


      print("======total=====")

      print(notes_tokens)

      sequence_len = sum(len(i) for i in notes_tokens)

      counter = counter+1
      if counter == 10:
        break
      data.append((keys_tokens, notes_tokens))
  return data

In [ ]:
train_data = []
valid_data = []
test_data = []

train_data = load_dataset(TRAIN_DIR)
# valid_data = load_dataset(VALID_DIR)
# test_data = load_dataset(TEST_DIR)

/content/drive/MyDrive/test2/8352_9782.abc
======total=====
[[58, 33, 18, 14, 17, 38, 18, 14, 17, 12, 35, 18, 14, 17, 12, 38, 11, 18, 14, 17, 12, 35, 11, 18, 14, 17, 12, 7131, 38, 14, 17, 35, 14, 17, 38, 11, 14, 17, 35, 11, 14, 17, 12, 7131, 33, 14, 17, 38, 14, 17, 35, 14, 17, 35, 11, 14, 17, 12, 60, 35, 11, 14, 17, 49146, 33, 14, 17, 38, 14, 17, 35, 14, 17, 35, 11, 14, 17, 12, 60, 35, 11, 14, 17, 49146, 33, 14, 17, 38, 14, 17, 35, 14, 17, 35, 11, 14, 17, 12, 60, 35, 11, 14, 17, 49146, 33, 14, 17, 38, 14, 17, 35, 14, 17, 35, 11, 14, 17, 12, 60, 35, 11, 14, 17, 49146, 33, 14, 17, 38, 14, 17, 35, 14, 17, 35, 11, 14, 17, 12, 60, 35, 11, 14, 17, 49146, 33, 14, 17, 38, 14, 17, 35, 14, 17, 35, 11, 14, 17, 12, 60, 35, 11, 14, 17, 12, 930, 220], [58, 33, 17, 38, 17, 35, 17, 38, 11, 17, 12, 35, 11, 17, 12, 7131, 33, 21, 38, 21, 35, 21, 38, 11, 21, 35, 11, 21, 12, 60, 930, 220], [58, 28, 32, 17, 37, 17, 35, 17, 35, 11, 17, 7131, 32, 14, 17, 12, 37, 14, 17, 12, 35, 14, 17, 7131, 32, 14, 17, 37, 1

In [ ]:
######
#BERT
#####
import torch
from torch.utils.data import Dataset


class ABCD_BERT(Dataset):
    def __init__(self, data,
                 context_bars_num=8,
                 target_bars_num=8,
                 bos_id=2,
                 eos_id=3,
                 is_test=False):

        self.notes = []
        self.keys = []

        for (keys, notes) in data:
            if notes is None:
                continue

            self.keys.append(keys)
            self.notes.append(notes)

        self.context_bars_num = context_bars_num
        self.target_bars_num = target_bars_num
        self.bos_id = bos_id
        self.eos_id = eos_id
        self.is_test = is_test

    def __len__(self):
        return len(self.keys)


    def __getitem__(self, idx):
        notes = self.notes[idx]
        keys = self.keys[idx]

        if not self.is_test:
            split_indx = 12

            # split notes to context (input for network) and target (that model must to generate)
            context_notes = notes[split_indx - self.context_bars_num : split_indx]
            target_notes = notes[split_indx: split_indx + self.target_bars_num]

        else:
            context_notes = notes
            target_notes = []

        context_tokens = [self.bos_id] + keys
        target_tokens = [self.bos_id]

        for bar in context_notes:
            context_tokens += bar

        for bar in target_notes:
            target_tokens += bar

        context_tokens += [self.eos_id]
        target_tokens += [self.eos_id]

        context_tokens = torch.tensor(context_tokens, dtype=torch.long)
        target_tokens = torch.tensor(target_tokens, dtype=torch.long)

        return {
            "input_ids": context_tokens,
            "decoder_input_ids": target_tokens,
            "labels": target_tokens,
        }
    def save_to_csv(self, file_path):
        data = []
        for idx in range(len(self)):
            sample = self[idx]
            data.append({
                "input_ids": " ".join(str(token.item()) for token in sample["input_ids"]),
                "decoder_input_ids": " ".join(str(token.item()) for token in sample["decoder_input_ids"]),
                "labels": " ".join(str(token.item()) for token in sample["labels"])
            })

        df = pd.DataFrame(data)
        df.to_csv(file_path, index=False)



In [ ]:
train_dataset_2 = ABCD(train_data)
# valid_dataset = ABCD(valid_data)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_function(samples):

    input_ids = [sample["input_ids"] for sample in samples]
    print(samples[0])
    decoder_input_ids = [sample["decoder_input_ids"] for sample in samples]
    labels = [sample["labels"] for sample in samples]

    input_ids_padded = pad_sequence(input_ids, batch_first=True)
    decoder_input_ids_padded = pad_sequence(decoder_input_ids, batch_first=True)
    labels_padded = pad_sequence(labels, batch_first=True)

    attention_mask = input_ids_padded != 0
    decoder_attention_mask = decoder_input_ids_padded != 0

    batch = {
        "input_ids": input_ids_padded,
        "decoder_input_ids": decoder_input_ids_padded,
        "labels": labels_padded,
        "attention_mask": attention_mask,
        "decoder_attention_mask": decoder_attention_mask,
    }

    return batch


In [ ]:
OUTPUT_DIR

'/content/drive/MyDrive/Music_project/output_BERT_checkpoints'

In [ ]:
from transformers import Trainer, TrainingArguments,TrainerCallback
from transformers import get_cosine_schedule_with_warmup
from transformers import DataCollatorForLanguageModeling


training_args = TrainingArguments(
     output_dir='/content/first_run',
     overwrite_output_dir=True,
     evaluation_strategy="epoch",
     gradient_accumulation_steps=8, # recheck this one

     num_train_epochs=50,
     per_device_train_batch_size=8,
     per_device_eval_batch_size=8,
     save_strategy = 'steps',
     save_steps=500,
     eval_steps=500,
     # logging_steps = 1,
     logging_strategy = 'epoch',
     fp16=True,
     report_to="wandb",  # enable logging to W&B
     run_name="bert-base-music_project",

     logging_dir='/content/first_run',
 )


class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("flos", None)
        if state.is_local_process_zero:
            print(logs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=abc_dataset,
    # eval_dataset=valid_dataset,
    data_collator= collate_function,
    callbacks=[PrinterCallback],  # We can either pass the callback class this way or an instance of it (MyCallback())

)

# Start training


In [ ]:
#Fifth Run
trainer.train()

Step,Training Loss
1554,0.029700
3109,0.005100
4664,0.001800
6219,0.000800
7774,0.000400
9329,0.000200
10884,0.000100
12439,0.000000
13993,0.000000
15548,0.000000


{'loss': 0.0297, 'learning_rate': 1.982551546391753e-05, 'epoch': 1.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0051, 'learning_rate': 1.9625128865979383e-05, 'epoch': 2.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


...

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0, 'learning_rate': 1.6219201030927834e-05, 'epoch': 19.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0, 'learning_rate': 1.6018943298969075e-05, 'epoch': 20.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


In [ ]:
#Fourth Run
trainer.train()

Step,Training Loss
157,5.846500
314,3.619400
472,1.484100
629,0.364400
786,0.088200
944,0.044300
1101,0.029000
1259,0.020800
1416,0.016000
1573,0.012900


{'loss': 5.8465, 'learning_rate': 1.5700000000000002e-05, 'epoch': 1.0}
{'loss': 3.6194, 'learning_rate': 1.998904336335509e-05, 'epoch': 2.0}
{'loss': 1.4841, 'learning_rate': 1.9937679191605964e-05, 'epoch': 3.0}
{'loss': 0.3644, 'learning_rate': 1.984521179060989e-05, 'epoch': 4.0}
{'loss': 0.0882, 'learning_rate': 1.9711832381924365e-05, 'epoch': 4.99}
{'loss': 0.0443, 'learning_rate': 1.9536860733321152e-05, 'epoch': 6.0}
{'loss': 0.029, 'learning_rate': 1.9323238012155125e-05, 'epoch': 7.0}
{'loss': 0.0208, 'learning_rate': 1.9069142925435335e-05, 'epoch': 8.0}
{'loss': 0.016, 'learning_rate': 1.8778846657551135e-05, 'epoch': 9.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0129, 'learning_rate': 1.845206968721005e-05, 'epoch': 10.0}
{'loss': 0.0104, 'learning_rate': 1.8087755429170473e-05, 'epoch': 11.0}
{'loss': 0.0088, 'learning_rate': 1.769202778528286e-05, 'epoch': 12.0}
{'loss': 0.0075, 'learning_rate': 1.7264335740162244e-05, 'epoch': 12.99}
{'loss': 0.0064, 'learning_rate': 1.6803447414783938e-05, 'epoch': 14.0}
{'loss': 0.0057, 'learning_rate': 1.631711006253251e-05, 'epoch': 15.0}
{'loss': 0.005, 'learning_rate': 1.580117729483068e-05, 'epoch': 16.0}
{'loss': 0.0045, 'learning_rate': 1.526432162877356e-05, 'epoch': 17.0}
{'loss': 0.004, 'learning_rate': 1.4705589951155008e-05, 'epoch': 18.0}
{'loss': 0.0036, 'learning_rate': 1.4123563174739036e-05, 'epoch': 19.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


...

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0012, 'learning_rate': 2.373980779190238e-06, 'epoch': 39.0}
{'loss': 0.0011, 'learning_rate': 1.970601171790616e-06, 'epoch': 40.0}
{'loss': 0.0011, 'learning_rate': 1.6032437411085711e-06, 'epoch': 41.0}
{'loss': 0.0011, 'learning_rate': 1.2707792273019049e-06, 'epoch': 42.0}
{'loss': 0.0011, 'learning_rate': 9.728216135571323e-07, 'epoch': 43.0}
{'loss': 0.001, 'learning_rate': 7.14379386755859e-07, 'epoch': 44.0}
{'loss': 0.001, 'learning_rate': 4.945237734282282e-07, 'epoch': 44.99}
{'loss': 0.001, 'learning_rate': 3.1314792140057395e-07, 'epoch': 46.0}
{'loss': 0.001, 'learning_rate': 1.7330064880545784e-07, 'epoch': 47.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.001, 'learning_rate': 7.378965336347188e-08, 'epoch': 48.0}
{'loss': 0.001, 'learning_rate': 1.6287654589922653e-08, 'epoch': 49.0}
{'loss': 0.001, 'learning_rate': 0.0, 'epoch': 49.88}
{'train_runtime': 7376.2989, 'train_samples_per_second': 68.259, 'train_steps_per_second': 1.064, 'total_flos': 9.048081898115194e+16, 'train_loss': 0.23280484066267682, 'epoch': 49.88}


TrainOutput(global_step=7850, training_loss=0.23280484066267682, metrics={'train_runtime': 7376.2989, 'train_samples_per_second': 68.259, 'train_steps_per_second': 1.064, 'total_flos': 9.048081898115194e+16, 'train_loss': 0.23280484066267682, 'epoch': 49.88})

In [ ]:
#Third Run
trainer.train()

Step,Training Loss,Validation Loss
200,7.355100,5.505111
400,3.473400,2.016948
600,1.253200,0.429838
800,0.066700,0.139001
1000,0.025400,0.109631
1200,0.015300,0.098248
1400,0.009000,0.092101
1600,0.006900,0.088578
1800,0.005500,0.086435
2000,0.004100,0.084204


{'loss': 9.5008, 'learning_rate': 8.6e-06, 'epoch': 0.99}
{'loss': 7.3551, 'learning_rate': 1.73e-05, 'epoch': 1.99}
{'eval_loss': 5.505110740661621, 'eval_runtime': 7.077, 'eval_samples_per_second': 195.422, 'eval_steps_per_second': 24.445, 'epoch': 2.3}
{'loss': 5.19, 'learning_rate': 1.999748234942507e-05, 'epoch': 2.99}
{'loss': 3.4734, 'learning_rate': 1.9984890974505383e-05, 'epoch': 3.99}
{'eval_loss': 2.0169477462768555, 'eval_runtime': 7.4168, 'eval_samples_per_second': 186.469, 'eval_steps_per_second': 23.326, 'epoch': 4.6}
{'loss': 2.2175, 'learning_rate': 1.9961729363458e-05, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 1.2532, 'learning_rate': 1.9928022035699166e-05, 'epoch': 6.0}
{'eval_loss': 0.42983782291412354, 'eval_runtime': 7.1236, 'eval_samples_per_second': 194.142, 'eval_steps_per_second': 24.285, 'epoch': 6.91}
{'loss': 0.557, 'learning_rate': 1.9883804674584312e-05, 'epoch': 7.0}
{'loss': 0.1924, 'learning_rate': 1.982912408963285e-05, 'epoch': 8.0}
{'loss': 0.0667, 'learning_rate': 1.9764845145447687e-05, 'epoch': 8.99}
{'eval_loss': 0.13900142908096313, 'eval_runtime': 7.4351, 'eval_samples_per_second': 186.008, 'eval_steps_per_second': 23.268, 'epoch': 9.21}
{'loss': 0.0371, 'learning_rate': 1.9689541163440347e-05, 'epoch': 9.99}
{'loss': 0.0254, 'learning_rate': 1.9603979609434666e-05, 'epoch': 10.99}
{'eval_loss': 0.10963056981563568, 'eval_runtime': 7.4455, 'eval_samples_per_second': 185.75, 'eval_steps_per_second': 23.236, 'epoch': 11.51}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


...

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0004, 'learning_rate': 1.8923962767615545e-07, 'epoch': 93.0}
{'loss': 0.0004, 'learning_rate': 1.3144283593192752e-07, 'epoch': 94.0}
{'eval_loss': 0.0730486586689949, 'eval_runtime': 7.4922, 'eval_samples_per_second': 184.593, 'eval_steps_per_second': 23.091, 'epoch': 94.39}
{'loss': 0.0004, 'learning_rate': 8.40931266576206e-08, 'epoch': 95.0}
{'loss': 0.0004, 'learning_rate': 4.7240625348735636e-08, 'epoch': 96.0}
{'eval_loss': 0.07308094203472137, 'eval_runtime': 7.2017, 'eval_samples_per_second': 192.037, 'eval_steps_per_second': 24.022, 'epoch': 96.69}
{'loss': 0.0004, 'learning_rate': 2.1166858367646092e-08, 'epoch': 96.99}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0004, 'learning_rate': 5.293115445467179e-09, 'epoch': 97.99}
{'eval_loss': 0.0730840414762497, 'eval_runtime': 7.0677, 'eval_samples_per_second': 195.679, 'eval_steps_per_second': 24.478, 'epoch': 98.99}
{'loss': 0.0004, 'learning_rate': 0.0, 'epoch': 98.99}
{'train_runtime': 9283.0026, 'train_samples_per_second': 59.873, 'train_steps_per_second': 0.926, 'total_flos': 1.496030941481132e+17, 'train_loss': 0.30288746738893, 'epoch': 98.99}


TrainOutput(global_step=8600, training_loss=0.30288746738893, metrics={'train_runtime': 9283.0026, 'train_samples_per_second': 59.873, 'train_steps_per_second': 0.926, 'total_flos': 1.496030941481132e+17, 'train_loss': 0.30288746738893, 'epoch': 98.99})

In [ ]:
#Second Run
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: adam-mourad1960 (music_project). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss
200,6.199700,5.350147
400,2.225500,1.906656
600,0.549400,0.511615
800,0.149000,0.217047
1000,0.101100,0.181315


{'loss': 10.6595, 'learning_rate': 2.1000000000000002e-06, 'epoch': 0.97}
{'loss': 9.6048, 'learning_rate': 4.3e-06, 'epoch': 1.98}
{'loss': 9.1835, 'learning_rate': 6.5000000000000004e-06, 'epoch': 2.99}
{'loss': 8.7154, 'learning_rate': 8.700000000000001e-06, 'epoch': 4.0}
{'loss': 8.6013, 'learning_rate': 1.0800000000000002e-05, 'epoch': 4.97}
{'loss': 7.6603, 'learning_rate': 1.3000000000000001e-05, 'epoch': 5.98}
{'loss': 7.0804, 'learning_rate': 1.5200000000000002e-05, 'epoch': 6.99}
{'loss': 6.4889, 'learning_rate': 1.7400000000000003e-05, 'epoch': 8.0}
{'loss': 6.1997, 'learning_rate': 1.95e-05, 'epoch': 8.97}
{'eval_loss': 5.350146770477295, 'eval_runtime': 4.3195, 'eval_samples_per_second': 320.177, 'eval_steps_per_second': 20.141, 'epoch': 9.2}
{'loss': 5.3507, 'learning_rate': 1.9980267284282718e-05, 'epoch': 9.98}
{'loss': 4.8314, 'learning_rate': 1.9896292772724142e-05, 'epoch': 10.99}
{'loss': 4.3184, 'learning_rate': 1.974692387082714e-05, 'epoch': 12.0}
{'loss': 4.0791

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 1.0956, 'learning_rate': 1.4457383557765385e-05, 'epoch': 22.99}
{'loss': 0.9336, 'learning_rate': 1.3715584763641345e-05, 'epoch': 24.0}
{'loss': 0.8207, 'learning_rate': 1.2984529248893081e-05, 'epoch': 24.97}
{'loss': 0.6598, 'learning_rate': 1.2199463578396688e-05, 'epoch': 25.98}
{'loss': 0.5494, 'learning_rate': 1.1399863921984151e-05, 'epoch': 26.99}
{'eval_loss': 0.5116150975227356, 'eval_runtime': 4.2557, 'eval_samples_per_second': 324.973, 'eval_steps_per_second': 20.443, 'epoch': 27.59}
{'loss': 0.4617, 'learning_rate': 1.0591014008951555e-05, 'epoch': 28.0}
{'loss': 0.4071, 'learning_rate': 9.815210950408703e-06, 'epoch': 28.97}
{'loss': 0.3298, 'learning_rate': 9.00373778573246e-06, 'epoch': 29.98}
{'loss': 0.282, 'learning_rate': 8.198847890328405e-06, 'epoch': 30.99}
{'loss': 0.2425, 'learning_rate': 7.4058599512249345e-06, 'epoch': 32.0}
{'loss': 0.2208, 'learning_rate': 6.664834894950232e-06, 'epoch': 32.97}
{'loss': 0.1854, 'learning_rate': 5.91013623160902e-

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0961, 'learning_rate': 1.7026900316098217e-07, 'epoch': 45.98}
{'loss': 0.0955, 'learning_rate': 5.3500806496741276e-08, 'epoch': 46.99}
{'loss': 0.0954, 'learning_rate': 2.458762615035193e-09, 'epoch': 48.0}
{'loss': 0.0963, 'learning_rate': 0.0, 'epoch': 48.28}
{'train_runtime': 2595.9894, 'train_samples_per_second': 107.05, 'train_steps_per_second': 0.404, 'total_flos': 8.114630532889133e+16, 'train_loss': 2.508532467399325, 'epoch': 48.28}


TrainOutput(global_step=1050, training_loss=2.508532467399325, metrics={'train_runtime': 2595.9894, 'train_samples_per_second': 107.05, 'train_steps_per_second': 0.404, 'total_flos': 8.114630532889133e+16, 'train_loss': 2.508532467399325, 'epoch': 48.28})

In [ ]:
#First RUN
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.010100,0.178636
1000,0.003200,0.158075
1500,0.001800,0.152992
2000,0.001200,0.151644
2500,0.000900,0.149198
3000,0.000700,0.148097


{'loss': 1.3521, 'learning_rate': 4.9e-05, 'epoch': 0.99}
{'loss': 0.2353, 'learning_rate': 4.7985507246376815e-05, 'epoch': 2.0}
{'loss': 0.0558, 'learning_rate': 4.698550724637682e-05, 'epoch': 2.99}
{'loss': 0.0282, 'learning_rate': 4.597101449275363e-05, 'epoch': 4.0}
{'loss': 0.0184, 'learning_rate': 4.497101449275363e-05, 'epoch': 4.99}
{'loss': 0.013, 'learning_rate': 4.395652173913043e-05, 'epoch': 6.0}
{'loss': 0.0101, 'learning_rate': 4.2956521739130435e-05, 'epoch': 6.99}
{'eval_loss': 0.178636372089386, 'eval_runtime': 5.9065, 'eval_samples_per_second': 188.267, 'eval_steps_per_second': 23.533, 'epoch': 7.19}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0079, 'learning_rate': 4.194202898550725e-05, 'epoch': 8.0}
{'loss': 0.0066, 'learning_rate': 4.094202898550725e-05, 'epoch': 8.99}
{'loss': 0.0054, 'learning_rate': 3.9927536231884064e-05, 'epoch': 10.0}
{'loss': 0.0047, 'learning_rate': 3.892753623188406e-05, 'epoch': 10.99}
{'loss': 0.0041, 'learning_rate': 3.7913043478260876e-05, 'epoch': 12.0}
{'loss': 0.0036, 'learning_rate': 3.691304347826087e-05, 'epoch': 12.99}
{'loss': 0.0032, 'learning_rate': 3.589855072463768e-05, 'epoch': 14.0}
{'eval_loss': 0.1580747365951538, 'eval_runtime': 5.7716, 'eval_samples_per_second': 192.668, 'eval_steps_per_second': 24.084, 'epoch': 14.39}


...

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0009, 'learning_rate': 1.373913043478261e-05, 'epoch': 36.0}
{'loss': 0.0009, 'learning_rate': 1.2739130434782608e-05, 'epoch': 36.99}
{'loss': 0.0008, 'learning_rate': 1.1724637681159421e-05, 'epoch': 38.0}
{'loss': 0.0008, 'learning_rate': 1.072463768115942e-05, 'epoch': 38.99}
{'loss': 0.0008, 'learning_rate': 9.710144927536233e-06, 'epoch': 40.0}
{'loss': 0.0008, 'learning_rate': 8.710144927536231e-06, 'epoch': 40.99}
{'loss': 0.0008, 'learning_rate': 7.695652173913044e-06, 'epoch': 42.0}
{'loss': 0.0007, 'learning_rate': 6.695652173913043e-06, 'epoch': 42.99}
{'eval_loss': 0.14809656143188477, 'eval_runtime': 5.6153, 'eval_samples_per_second': 198.031, 'eval_steps_per_second': 24.754, 'epoch': 43.17}


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'loss': 0.0007, 'learning_rate': 5.681159420289855e-06, 'epoch': 44.0}
{'loss': 0.0007, 'learning_rate': 4.6811594202898555e-06, 'epoch': 44.99}
{'loss': 0.0007, 'learning_rate': 3.666666666666667e-06, 'epoch': 46.0}
{'loss': 0.0007, 'learning_rate': 2.666666666666667e-06, 'epoch': 46.99}
{'loss': 0.0007, 'learning_rate': 1.6521739130434782e-06, 'epoch': 48.0}
{'loss': 0.0007, 'learning_rate': 6.521739130434782e-07, 'epoch': 48.99}
{'loss': 0.0007, 'learning_rate': 0.0, 'epoch': 49.64}
{'train_runtime': 3295.4549, 'train_samples_per_second': 67.457, 'train_steps_per_second': 1.047, 'total_flos': 4.15398998828304e+16, 'train_loss': 0.035935078265442365, 'epoch': 49.64}


TrainOutput(global_step=3450, training_loss=0.035935078265442365, metrics={'train_runtime': 3295.4549, 'train_samples_per_second': 67.457, 'train_steps_per_second': 1.047, 'total_flos': 4.15398998828304e+16, 'train_loss': 0.035935078265442365, 'epoch': 49.64})

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 15 13:12:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Fourth RUN
wandb.finish()


train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▆▆██████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,49.88
train/global_step,7850


In [ ]:
# Third RUN
wandb.finish()


eval/loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▂▂▂▂▂▂▁▁▂▁▂▁▂▂▂▄▁▁▂▂▂▂█▂▂▂▇▂▂▂▄▂▁▂▂▂▁▂▁
eval/samples_per_second,▇▆▇▆▆▇▇█▇▇▇▇▇▆▇▇▄▇▇▇▇▇▆▁▇▇▇▂▇▇▇▄▇█▇▆▇█▆▇
eval/steps_per_second,▇▆▇▆▆▇▇█▇▇▇▇▇▆▇▇▄▇▇▇▇▇▆▁▇▇▇▂▇▇▇▄▇█▇▆▇█▆▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▄███████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
# SECOND RUN
wandb.finish()



eval/loss,█▃▁▁▁
eval/runtime,█▆▅▃▁
eval/samples_per_second,▁▃▄▆█
eval/steps_per_second,▁▃▄▆█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train/learning_rate,▂▃▃▄▅▆▇██████▇▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁
train/loss,█▇▇▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
# FIRST RUN
wandb.finish()



eval/loss,█▃▂▂▁▁
eval/runtime,█▅█▁▂▂
eval/samples_per_second,▁▄▁█▇▇
eval/steps_per_second,▁▄▁█▇▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
